In [2]:
import requests
import json
import re
import os


In [13]:
# get file IDs and matching metadata in 

files_endpt = "https://api.gdc.cancer.gov/files"

# filtering for cases that are primary site pancreas, and have methylation data
filters = {
    "op": "and",
    "content":[
        {
        "op": "in",
        "content":{
            "field": "cases.primary_site",
            "value": ["pancreas"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "files.data_category",
            "value": ["dna methylation"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "files.data_type",

            ### "Masked Intensities" selects for .IDAT raw methylation files
            ### "Methylation Beta Values" selects for .txt beta arrays files
            "value": ["Methylation Beta Value"]
            }
        }
    ]
}

### could add more data catagories here (bmi, sex... ) https://docs.gdc.cancer.gov/API/Users_Guide/Appendix_A_Available_Fields/#file-fields
fields = [
    "file_id",
    "file_name",
    "platform",

    "cases.case_id",

    "cases.diagnoses.created_datetime",

    "cases.demographic.vital_status",
    "cases.demographic.days_to_birth",
    "cases.demographic.days_to_death",

    "cases.samples.sample_id",
    "cases.samples.created_datetime"
    ]
fields = ",".join(fields)

params = {
    "filters": json.dumps(filters),
    "fields": fields,
    "format": "JSON",
    "size": "1000"
    }

response = requests.get(files_endpt, params = params)


{'id': '14218616-8764-4718-93d6-3d95fd424c1e', 'cases': [{'case_id': '55b590d7-fecf-422a-bc3f-51e118daf7a9', 'diagnoses': [{'created_datetime': '2018-12-03T12:05:16.846188-06:00'}], 'samples': [{'sample_id': '9257b022-9307-4bb8-8295-d2c4b228ab81', 'created_datetime': '2018-06-12T14:47:35.657768-05:00'}], 'demographic': {'vital_status': 'Dead', 'days_to_death': 881, 'days_to_birth': -23457}}], 'file_name': '4f01e6a4-68b0-4298-81c2-b89b0a950379.methylation_array.sesame.level3betas.txt', 'file_id': '14218616-8764-4718-93d6-3d95fd424c1e', 'platform': 'Illumina Methylation Epic'}
{'id': 'ad8dadac-cc82-46d8-b49a-040ac096bc46', 'cases': [{'case_id': '55b590d7-fecf-422a-bc3f-51e118daf7a9', 'diagnoses': [{'created_datetime': '2018-12-03T12:05:16.846188-06:00'}], 'samples': [{'sample_id': '3bb4cd96-e5c9-4c2f-8633-2c8b47c74905', 'created_datetime': '2018-06-12T14:47:35.657768-05:00'}], 'demographic': {'vital_status': 'Dead', 'days_to_death': 881, 'days_to_birth': -23457}}], 'file_name': '8451f4a1

In [18]:
data_endpt = "https://api.gdc.cancer.gov/data"

ids = [
    "b658d635-258a-4f6f-8377-767a43771fe4",
    "3968213d-b293-4b3d-8033-5b5a0ca07b6c"
    ]

params = {"ids": ids}

response = requests.post(data_endpt,
                        data = json.dumps(params),
                        headers={
                            "Content-Type": "application/json"
                            })

response_head_cd = response.headers["Content-Disposition"]

file_name = re.findall("filename=(.+)", response_head_cd)[0]

save_name = "/Users/zacksiegfried/Documents/methylspan/beta_arrays/" + file_name
with open(save_name, "wb") as output_file:
    output_file.write(response.content)